In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os

def crawl_jd_search_results(keyword, max_results):
    # Construct the initial search URL with the provided keyword
    url = f'https://search.jd.com/Search?keyword={keyword}&enc=utf-8&page=1'

    # Set headers to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
    }

    # Create an empty list to store the search results
    results = []

    # Initialize the page counter
    page = 1

    # Continue crawling until the desired number of results is reached
    while len(results) < max_results:
        # Send an HTTP GET request to fetch the page content
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the product items from the search results
        products = soup.select('.gl-item')

        # Process each product item
        for product in products:
            # Extract the product name
            name = product.select_one('.p-name em').text.strip()

            # Extract the product price
            price = product.select_one('.p-price').text.strip()

            # Create a dictionary to store the product information
            product_info = {'Name': name, 'Price': price}

            # Add the product information to the results list
            results.append(product_info)

            # Break the loop if the desired number of results is reached
            if len(results) >= max_results:
                break

        # Increment the page counter
        page += 1

        # Update the URL to fetch the next page of search results
        url = f'https://search.jd.com/Search?keyword={keyword}&enc=utf-8&page={page}'

    # Return the search results, limited to the desired number of results
    return results[:max_results]

def save_to_csv(data, filename):
    # Extract the keys from the first dictionary in the data list
    keys = data[0].keys()

    # Open the CSV file in write mode
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        # Create a CSV writer object
        writer = csv.DictWriter(file, fieldnames=keys)

        # Write the header row to the CSV file
        writer.writeheader()

        # Write the data rows to the CSV file
        writer.writerows(data)

def main():
    # Set the keyword for the search
    keyword = '粉色键盘'

    # Set the maximum number of search results to retrieve
    max_results = 500

    # Set the filename for saving the search results
    filename = os.path.join(os.path.expanduser("~"), 'Desktop', '粉色键盘.csv')

    # Get the search results
    search_results = crawl_jd_search_results(keyword, max_results)

    # Save the search results to a CSV file
    save_to_csv(search_results, filename)
    print(f'Saved {len(search_results)} search results to {filename}')

if __name__ == '__main__':
    main()
